In [1]:
%load_ext autoreload
%autoreload 3
%reload_ext autoreload
import pandas as pd
import openpyxl
import re
from datetime import datetime
from pathlib import Path
import os
from io import BytesIO
from azure.storage.blob import BlobServiceClient
from planification.pool import *

import numpy as np
from datetime import datetime, timedelta

In [7]:
import hashlib

hashlib.sha256("your_actual_password".encode()).hexdigest()

'1f8bfc350cd6e77b82f5116e2503f517a13a197695f825a339e5be2cae8608bf'

In [9]:
hashlib.sha256("komhive".encode()).hexdigest()

'7f75db3f36562e7721f0e5137b045f2a3a39b76dbe19af40d2f67a4dd95aedcc'

In [18]:
import plotly.express as px

In [34]:
partition_date = "y=2024/m=07"
blob_service_client = BlobServiceClient(
    account_url=os.environ["AZURE_ACCOUNT_URL"],
    credential=os.environ["AZURE_SAS_TOKEN"],
)

container_client = blob_service_client.get_container_client(
    os.environ["AZURE_CONTAINER_NAME"]
)
blob_list = container_client.list_blobs(
    name_starts_with=f"{os.environ['AZURE_PREFIX']}/HSEC/{partition_date}"
)
blob_list = [f.name for f in blob_list]
# assert blob_list.__len__() <= 1, "There should be only one file per month"
file = blob_list[0]

blob_client = blob_service_client.get_blob_client(
    container=os.environ["AZURE_CONTAINER_NAME"],
    blob=file,
)
blob_data = blob_client.download_blob()
blob_data = BytesIO(blob_data.readall())

In [40]:
adherence_df = pd.read_excel(
    blob_data,
    skiprows=7,
    sheet_name="3D",
    usecols="B:K",
)
adherence_df = adherence_df.assign(
    adherence=adherence_df["Estado Informe Final"].map(
        lambda x: {
            "NO APLICA 3D": "VIGENTE",
            "SIN 3D": "NO VIGENTE",
            "No Vigente": "NO VIGENTE",
        }.get(x, x)
    )
).rename(
    columns={"Rut/Passport": "RUT_PERSONA", "Estado Informe Final": "ESTADO_FINAL"}
)
adherence_df

,ESTADO_FINAL,RUT_PERSONA,Nombre,Nombre Empleador,SAP Empleador,Contratista o Subcontratista?,Posicion PMF,Descripción Posición,Aplicabilidad,Fecha Caducidad 3D,adherence
0,NO APLICA 3D,10071845-6,Eric Norambuena,KOMATSU CHILE S A,30124989,Contratista,Inspector,Inspector Técnico,NO APLICA,NO APLICA 3D,VIGENTE
1,NO APLICA 3D,11192647-6,Sergio Dinamarca,SERVICIOS AUTOMOTRIZ CLEAN CAR S.A.,30124989,Subcontratista,Operador,Conductor,NO APLICA,NO APLICA 3D,VIGENTE
2,NO APLICA 3D,11599361-5,Claudio Patricio Diaz Orellana,SERVICIOS AUTOMOTRIZ CLEAN CAR S.A.,30124989,Subcontratista,Operador,Conductor,NO APLICA,NO APLICA 3D,VIGENTE
3,SIN 3D,11940448-7,Zenon Pasten,DISTRIBUIDORA CUMMINS,30124989,Subcontratista,Operador,,ACTUALIZACION DESCRIPCION PMF,SIN 3D,NO VIGENTE
4,VIGENTE,12052942-0,CRISTIAN GONZALO SALAZAR ESPINOZA,KOMATSU CHILE S A,30124989,Contratista,Supervisor,Jefe de Mantención,APLICA,2026-01-24 00:00:00,VIGENTE
...,...,...,...,...,...,...,...,...,...,...,...
101,SIN 3D,20707214-1,JONATHAN JESUS ROJAS CANALES,KOMATSU CHILE S A,30124989,Contratista,Operador,Mecánico 2-A,APLICA,,NO VIGENTE
102,VIGENTE,20751525-6,JORGE ALBERTO TAPIA VARAS,DISTRIBUIDORA CUMMINS,30124989,Subcontratista,Operador,Mecanico 2-B,APLICA,2025-05-29 00:00:00,VIGENTE
103,VIGENTE,20774775-0,BRAYHAN ALEXIS GUTIERREZ GONZALEZ,KOMATSU CHILE S A,30124989,Contratista,Posición,Electromecánico 1-F,APLICA,2026-07-03 00:00:00,VIGENTE
104,SIN 3D,21774098-3,Oscar Cortes,DISTRIBUIDORA CUMMINS,30124989,Subcontratista,Supervisor,SUPERVISOR DE FAENA,APLICA,,NO VIGENTE


In [48]:
performance_df = (
    pd.read_excel(
        blob_data,
        skiprows=7,
        sheet_name="3D",
        usecols="M:V",
    )
    .rename(
        columns={
            "Rut/Passport.1": "RUT_PASSPORT",
            "Estado Informe Final": "ESTADO_FINAL",
        }
    )
    .drop(
        columns=[
            "Nombre.1",
            "Nombre Empleador.1",
            "SAP Empleador.1",
            "Contratista o Subcontratista?.1",
        ]
    )
    .dropna(subset=["RUT_PASSPORT"])
)
performance_df

,Resultado Final,Perfil 3D,RUT_PASSPORT,Resultado Dimensión Psicolaboral,Resultado Dimensión Seguridad,Resultado Dimensión Técnica
0,COMPETENTE,SUPERVISOR,12052942-0,COMPETENTE,COMPETENTE,COMPETENTE
1,COMPETENTE CON OBSERVACIONES,TÉCNICO,13744790-8,COMPETENTE,COMPETENTE,COMPETENTE CON OBSERVACIONES
2,COMPETENTE CON OBSERVACIONES,TÉCNICO,15033438-1,COMPETENTE CON OBSERVACIONES,COMPETENTE,COMPETENTE CON OBSERVACIONES
3,COMPETENTE,TÉCNICO,16468103-3,COMPETENTE CON OBSERVACIONES,COMPETENTE,COMPETENTE
4,CON OBSERVACIONES,TÉCNICO,16504327-8,COMPETENTE CON OBSERVACIONES,CON OBSERVACIONES,COMPETENTE
5,COMPETENTE CON OBSERVACIONES,TÉCNICO,16681703-K,COMPETENTE CON OBSERVACIONES,COMPETENTE,COMPETENTE CON OBSERVACIONES
6,COMPETENTE,TÉCNICO,16704464-6,COMPETENTE CON OBSERVACIONES,COMPETENTE,COMPETENTE
7,COMPETENTE,TÉCNICO,16893572-2,COMPETENTE,COMPETENTE,COMPETENTE CON OBSERVACIONES
8,COMPETENTE,TÉCNICO,17364687-9,COMPETENTE CON OBSERVACIONES,COMPETENTE,COMPETENTE
9,COMPETENTE,TÉCNICO,17626725-9,COMPETENTE,COMPETENTE,COMPETENTE


In [74]:
ceim_df = pd.read_excel(
    "/home/cecilvega/Downloads/Gestion_Resultados_02_08_2024.xlsx"
).rename(columns={"RUT_PERSONA": "RUT_PASSPORT"})
ceim_df["FECHA_PRESENCIAL"] = pd.to_datetime(
    ceim_df["FECHA_PRESENCIAL"], format="%d/%m/%Y"
)
ceim_df = ceim_df.sort_values(["RUT_PASSPORT", "FECHA_PRESENCIAL"]).drop_duplicates(
    subset=["RUT_PASSPORT"], keep="last"
)

,ID,APLICA_3D,RUT_PASSPORT,NOMBRE_PERSONA,EMAIL,TELEFONO,CARGO_CONTRATO,FECHA_ONLINE,FECHA_PRESENCIAL,EMPRESA,...,PUNTAJE_GESTION,RESULTADO_GESTION,FECHA_EVALUACION_GESTION,FECHA_VENCIMIENTO_GESTION,RESULTADO_CERTIFICACION,FECHA_VENCIMIENTO_CERTIFICACION,ESTADO_FINAL,ESTADO_INFORME,FECHA_CADUCIDAD,NUMERO_CONTRATO
262,55053,SI,13744790-8,MARIO FRANCISCO SANDOVAL CORTES,MARIO.SANDOVAL@GLOBAL.KOMATSU,9999999,MECÁNICO 1-E,28/06/2024,2024-07-02,KOMATSU CHILE S.A.,...,NO APLICA,NO APLICA,NO APLICA,NO APLICA,NO APLICA,NO APLICA,COMPETENTE CON OBSERVACIONES,INFORME APLICA DISPONIBLE,02/01/2025,NaN


In [78]:
ceim_df

,RUT_PASSPORT,ESTADO_FINAL
58,10394739-1,COMPETENTE
1,10756662-7,COMPETENTE
194,11467788-4,PENDIENTE
197,11508808-4,COMPETENTE
37,11940448-7,COMPETENTE
...,...,...
230,24058348-8,PENDIENTE
205,24381239-9,COMPETENTE CON OBSERVACIONES
209,24540820-k,PENDIENTE
206,8561594-7,PENDIENTE


In [68]:
ceim_df.FECHA_PRESENCIAL.iloc[0]

'05/03/2022'

In [80]:
ceim_df = pd.read_excel(
    "/home/cecilvega/Downloads/Gestion_Resultados_02_08_2024.xlsx"
).rename(columns={"RUT_PERSONA": "RUT_PASSPORT"})
ceim_df = pd.read_excel(
    "/home/cecilvega/Downloads/Gestion_Resultados_02_08_2024.xlsx"
).rename(columns={"RUT_PERSONA": "RUT_PASSPORT"})
ceim_df["FECHA_PRESENCIAL"] = pd.to_datetime(
    ceim_df["FECHA_PRESENCIAL"], format="%d/%m/%Y"
)
ceim_df = ceim_df.sort_values(["RUT_PASSPORT", "FECHA_PRESENCIAL"]).drop_duplicates(
    subset=["RUT_PASSPORT"], keep="last"
)
ceim_df = ceim_df[["RUT_PASSPORT", "ESTADO_FINAL"]]
ceim_df = ceim_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
ceim_df

df = pd.merge(performance_df, ceim_df, on="RUT_PASSPORT", how="left")
df.loc[df["Resultado Final"] == df["ESTADO_FINAL"]]

,Resultado Final,Perfil 3D,RUT_PASSPORT,Resultado Dimensión Psicolaboral,Resultado Dimensión Seguridad,Resultado Dimensión Técnica,ESTADO_FINAL
0,COMPETENTE,SUPERVISOR,12052942-0,COMPETENTE,COMPETENTE,COMPETENTE,COMPETENTE
1,COMPETENTE CON OBSERVACIONES,TÉCNICO,13744790-8,COMPETENTE,COMPETENTE,COMPETENTE CON OBSERVACIONES,COMPETENTE CON OBSERVACIONES
2,COMPETENTE CON OBSERVACIONES,TÉCNICO,15033438-1,COMPETENTE CON OBSERVACIONES,COMPETENTE,COMPETENTE CON OBSERVACIONES,COMPETENTE CON OBSERVACIONES
3,COMPETENTE,TÉCNICO,16468103-3,COMPETENTE CON OBSERVACIONES,COMPETENTE,COMPETENTE,COMPETENTE
4,CON OBSERVACIONES,TÉCNICO,16504327-8,COMPETENTE CON OBSERVACIONES,CON OBSERVACIONES,COMPETENTE,CON OBSERVACIONES
5,COMPETENTE CON OBSERVACIONES,TÉCNICO,16681703-K,COMPETENTE CON OBSERVACIONES,COMPETENTE,COMPETENTE CON OBSERVACIONES,COMPETENTE CON OBSERVACIONES
6,COMPETENTE,TÉCNICO,16704464-6,COMPETENTE CON OBSERVACIONES,COMPETENTE,COMPETENTE,COMPETENTE
7,COMPETENTE,TÉCNICO,16893572-2,COMPETENTE,COMPETENTE,COMPETENTE CON OBSERVACIONES,COMPETENTE
8,COMPETENTE,TÉCNICO,17364687-9,COMPETENTE CON OBSERVACIONES,COMPETENTE,COMPETENTE,COMPETENTE
9,COMPETENTE,TÉCNICO,17626725-9,COMPETENTE,COMPETENTE,COMPETENTE,COMPETENTE


In [57]:
ceim_df.RUT_PASSPORT.iloc[0]

'19041711-5         '

In [ ]:
pd.merge(adherence_df, adherence_df, on="Estado Informe Final", how="left")

In [20]:
px.data.medals_long()

,nation,medal,count
0,South Korea,gold,24
1,China,gold,10
2,Canada,gold,9
3,South Korea,silver,13
4,China,silver,15
5,Canada,silver,12
6,South Korea,bronze,11
7,China,bronze,8
8,Canada,bronze,12


In [30]:
adherence_df.groupby(["adherence", "Estado Informe Final"]).size().reset_index(
    name="count"
)

,adherence,Estado Informe Final,count
0,NO VIGENTE,NO VIGENTE,8
1,NO VIGENTE,SIN 3D,40
2,VIGENTE,NO APLICA 3D,36
3,VIGENTE,VIGENTE,22


In [13]:
adherence_df["Estado Informe Final"].unique()

array(['NO APLICA 3D', 'SIN 3D', 'VIGENTE', 'NO VIGENTE'], dtype=object)

In [2]:
available_components = [
    "bp",
    # "cd",
    "mt",
    # "st", "cms", "cl", "mp"
]
cc_df = read_cc()
cc_df = cc_df.loc[
    ~(
        (cc_df["component_code"] == "mt")
        & ~(cc_df["subcomponente"].str.contains("MOTOR TRACCIÓN"))
    )
]
cc_df = cc_df.loc[
    ~(
        (cc_df["component_code"] == "mt")
        & ~(cc_df["subcomponente"].str.contains("MOTOR TRACCIÓN"))
    )
]
pool_proj_df = read_base_pool_proj().drop(columns=["changeout_date"])
cc_df = cc_df.loc[cc_df["component_code"].isin(available_components)].reset_index(
    drop=True
)
pool_proj_df = pool_proj_df.loc[
    pool_proj_df["component_code"].isin(available_components)
].reset_index(drop=True)

/home/cecilvega/DataspellProjects/koapp/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
generate_pool_projection(cc_df, pool_proj_df)

equipo                                 322
componente                          Blower
subcomponente                       Blower
position                         IZQUIERDO
component_serial               #EN12040115
changeout_week                    2024-W28
changeout_date         2024-07-09 00:00:00
component_hours                     4490.0
tbo_hours                          20000.0
component_code                          bp
pool_changeout_type                      P
Name: 4, dtype: object
equipo                                 320
componente                          Blower
subcomponente                       Blower
position                         IZQUIERDO
component_serial               #EE13101272
changeout_week                    2024-W29
changeout_date         2024-07-15 00:00:00
component_hours                    13291.0
tbo_hours                          20000.0
component_code                          bp
pool_changeout_type                      P
Name: 5, dtype: object
equipo  

,pool_slot,changeout_week,equipo,component_serial,arrival_week,arrival_date,component_code,componente,subcomponente,position,changeout_date,component_hours,tbo_hours,pool_changeout_type
0,1,2023-W27,856,#EG08090003,2023-W40,2023-10-02,bp,Blower,Blower,DERECHO,2023-07-03,5726.0,20000.0,P
1,2,2023-W27,881,#EN12040115,2023-W34,2023-08-21,bp,Blower,Blower,IZQUIERDO,2023-07-05,24180.0,20000.0,P
2,3,2023-W27,875,#EN11110073,2023-W38,2023-09-18,bp,Blower,Blower,IZQUIERDO,2023-07-09,23790.0,20000.0,P
3,4,2023-W28,883,#EN12090007,2023-W38,2023-09-18,bp,Blower,Blower,DERECHO,2023-07-14,17676.0,20000.0,P
4,5,2023-W29,870,#EE14010117,2023-W42,2023-10-16,bp,Blower,Blower,DERECHO,2023-07-17,16263.0,20000.0,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,13,2024-W30,398,#EE16030252,2024-W37,2024-09-14,bp,Blower,Blower,DERECHO,2024-07-25,7398.0,20000.0,P
70,1,2024-W30,849,#WX15030003T,2024-W41,2024-10-07,mt,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,2024-07-25,23208.0,24000.0,P
71,2,2024-W30,871,#EE13111400,2024-W38,2024-09-16,bp,Blower,Blower,DERECHO,2024-07-27,18412.0,20000.0,P
72,8,2024-W31,285,#W08031004,2024-W41,2024-10-11,mt,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,2024-07-29,28605.0,24000.0,P


In [8]:
merge_columns = ["equipo", "component_code", "component_serial", "changeout_week"]

pool_slots_df = pd.merge(
    pool_proj_df,  # .query("component_code == 'mt'"),
    cc_df,
    on=merge_columns,
    how="left",
    suffixes=("_proj", ""),
)
pool_slots_df = pool_slots_df.assign(
    pool_changeout_type=np.where(
        pool_slots_df["pool_changeout_type"].isnull(),
        pool_slots_df["pool_changeout_type_proj"],
        pool_slots_df["pool_changeout_type"],
    ),
).drop(columns="pool_changeout_type_proj")
# assert pool_slots_df["changeout_date"].notnull().all()
pool_slots_df = pool_slots_df.dropna(subset=["changeout_date"])
# New changes that were not added into the pool
new_cc_df = (
    pd.merge(
        cc_df.loc[cc_df["changeout_date"] >= datetime(2024, 6, 1)],
        pool_proj_df[merge_columns],
        on=merge_columns,
        how="left",
        indicator=True,
    )
    .query("_merge == 'left_only'")
    .drop(columns="_merge")
    .assign(pool_changeout_type=lambda x: x["pool_changeout_type"].fillna("P"))
)

In [9]:
def add_arrival_date(df: pd.DataFrame):
    df = df.assign(
        ohv_normal=df["component_code"].map(
            lambda x: {
                "bp": 51,
                "cd": 46,
                "st": 65,
                "cms": 64,
                "mt": 74,
                "cl": 75,
                "mp": 64,
            }[x]
        ),
        ohv_unplanned=df["component_code"].map(
            lambda x: {
                "bp": 101,
                "cd": 96,
                "st": 125,
                "cms": 124,
                "mt": 134,
                "cl": 135,
                "mp": 114,
            }[x]
        ),
    )
    df = df.assign(
        arrival_date=df["changeout_date"]
        + np.where(
            df["pool_changeout_type"] == "P",
            pd.to_timedelta(df["ohv_normal"], "D"),
            pd.to_timedelta(df["ohv_unplanned"], "D"),
        )
    )
    df = df.assign(arrival_week=df["arrival_date"].dt.strftime("%G-W%V")).drop(
        columns=["ohv_normal", "ohv_unplanned"]
    )
    return df


def find_available_pool_slot(pool_slots_df, changeout_date):
    # Group by pool_slot and find the latest event for each
    latest_events = (
        pool_slots_df.groupby("pool_slot")[
            ["pool_slot", "changeout_date", "arrival_date"]
        ]
        .apply(
            lambda x: x.loc[
                (
                    x["changeout_date"].idxmax()
                    if not x["changeout_date"].isna().all()
                    else x["arrival_date"].idxmax()
                )
            ]
        )
        .reset_index(drop=True)
    )

    # Filter for available slots
    available_slots = latest_events[
        (
            (latest_events["changeout_date"] < changeout_date)
            & (latest_events["arrival_date"] < changeout_date)
        )
        | (latest_events["arrival_date"] < changeout_date)
    ]

    if available_slots.empty:
        return None

    return available_slots  # .loc[available_slots["days_unchanged"].idxmax()]


def find_most_time_unchanged_slot(df, changeout_date):
    # Find the slot with the most time unchanged
    df["days_unchanged"] = (changeout_date - df["arrival_date"]).dt.days
    return df.loc[df["days_unchanged"].idxmax()]


def allocate_components(cc_df, pool_slots_df):
    # Sort cc_df by changeout_date
    cc_df = cc_df.sort_values("changeout_date")

    for _, changeout in cc_df.iterrows():
        available_slots_df = find_available_pool_slot(
            pool_slots_df, changeout["changeout_date"]
        )
        print(changeout)
        available_slot = find_most_time_unchanged_slot(
            available_slots_df, changeout["changeout_date"]
        )
        if available_slot is None:
            print(
                f"Unable to find an available pool slot for changeout on {changeout['changeout_date']}"
            )
            # return pool_slots_df
        # Add new row to pool_slots_df
        new_row = changeout.copy()
        new_row["pool_slot"] = available_slot["pool_slot"]
        new_row["arrival_date"] = pd.NaT  # This will be set when the component returns

        new_row = pd.DataFrame([new_row])
        new_row = add_arrival_date(new_row)

        pool_slots_df = pd.concat([pool_slots_df, new_row], ignore_index=True)

        # Sort the dataframe to maintain chronological order
        pool_slots_df = pool_slots_df.sort_values("changeout_date")
    return pool_slots_df

In [5]:
pool_proj_df

,pool_slot,changeout_week,equipo,component_serial,arrival_week,pool_changeout_type,arrival_date,component_code
0,1,2023-W27,856,#EG08090003,2023-W40,P,2023-10-02,bp
1,2,2023-W27,881,#EN12040115,2023-W34,P,2023-08-21,bp
2,3,2023-W27,875,#EN11110073,2023-W38,P,2023-09-18,bp
3,4,2023-W28,883,#EN12090007,2023-W38,P,2023-09-18,bp
4,6,2023-W29,883,#EE06050200-1,2023-W42,P,2023-10-16,bp
...,...,...,...,...,...,...,...,...
96,3,2024-W15,864,#WX14030012T,2024-W34,I,2024-08-19,mt
97,4,2024-W17,399,#W11110013,2024-W36,I,2024-09-02,mt
98,5,2024-W18,397,#WX1712Y002T,2024-W37,I,2024-09-09,mt
99,6,2024-W21,856,#WX14020007T,2024-W39,I,2024-09-23,mt


In [10]:
df = allocate_components(new_cc_df, pool_slots_df)

equipo                                 322
componente                          Blower
subcomponente                       Blower
position                         IZQUIERDO
component_serial               #EN12040115
changeout_week                    2024-W28
changeout_date         2024-07-09 00:00:00
component_hours                     4490.0
tbo_hours                          20000.0
component_code                          bp
pool_changeout_type                      P
Name: 4, dtype: object
equipo                                 320
componente                          Blower
subcomponente                       Blower
position                         IZQUIERDO
component_serial               #EE13101272
changeout_week                    2024-W29
changeout_date         2024-07-15 00:00:00
component_hours                    13291.0
tbo_hours                          20000.0
component_code                          bp
pool_changeout_type                      P
Name: 5, dtype: object
equipo  

In [2]:
arrivals_df = read_pool_component_arrivals()

In [63]:
cc_df = read_cc()
# cc_df = cc_df.query("component_code == 'mt'")
cc_df = cc_df.loc[
    ~(
        (cc_df["component_code"] == "mt")
        & ~(cc_df["subcomponente"].str.contains("MOTOR TRACCIÓN"))
    )
]
cc_df

/home/cecilvega/DataspellProjects/koapp/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,equipo,componente,subcomponente,position,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type
0,862,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#W11110015,2019-W1,2019-01-01,14025.0,24000.0,mt,None
4,860,Blower,Blower,IZQUIERDO,#EE13111400,2019-W1,2019-01-01,26007.0,20000.0,bp,None
5,859,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,#WX15020001T,2019-W1,2019-01-04,25017.0,24000.0,mt,None
9,866,Blower,Blower,DERECHO,#EE07040188,2019-W2,2019-01-10,12558.0,20000.0,bp,None
10,875,Módulo_Potencia,MOTOR,UNICO,#66302957,2019-W2,2019-01-13,9457.0,16500.0,mp,None
...,...,...,...,...,...,...,...,...,...,...,...
2717,864,Cilindro_Dirección,Cilindro Dirección,IZQUIERDO,#EMWJ361,2024-W31,2024-08-01,13592.0,12000.0,cd,None
2718,850,CMS,Suspension Delantera,DERECHO,#201-1,2024-W31,2024-08-04,4724.0,15000.0,cms,None
2719,850,CMS,MASA,DERECHO,#LNTC11,2024-W31,2024-08-04,4724.0,15000.0,cms,None
2720,850,CMS,Freno Servicio Delanteros,DERECHO,#162,2024-W31,2024-08-04,4724.0,15000.0,cms,None


In [64]:
pool_proj_df = (
    read_base_pool_proj().drop(columns=["changeout_date"])
    # .rename(
    #     columns={
    #         "arrival_date": "arrival_date_archived",
    #         "arrival_week": "arrival_week_archived",
    #     }
    # )
)
pool_proj_df.query("pool_slot=='6' & component_code=='mt'")

,pool_slot,changeout_week,equipo,component_serial,arrival_week,pool_changeout_type,arrival_date,component_code
5,6,2023-W49,322,#WX14100052T,2024-W16,I,2024-04-15,mt
17,6,2024-W21,856,#WX14020007T,2024-W39,I,2024-09-23,mt


In [65]:
merge_columns = ["equipo", "component_code", "component_serial", "changeout_week"]

pool_slots_df = pd.merge(
    pool_proj_df,  # .query("component_code == 'mt'"),
    cc_df,
    on=merge_columns,
    how="left",
    suffixes=("_proj", ""),
)
pool_slots_df = pool_slots_df.assign(
    pool_changeout_type=np.where(
        pool_slots_df["pool_changeout_type"].isnull(),
        pool_slots_df["pool_changeout_type_proj"],
        pool_slots_df["pool_changeout_type"],
    ),
).drop(columns="pool_changeout_type_proj")
# assert pool_slots_df["changeout_date"].notnull().all()
pool_slots_df = pool_slots_df.dropna(subset=["changeout_date"])
# New changes that were not added into the pool
new_cc_df = (
    pd.merge(
        cc_df.loc[cc_df["changeout_date"] >= datetime(2024, 6, 1)],
        pool_proj_df[merge_columns],
        on=merge_columns,
        how="left",
        indicator=True,
    )
    .query("_merge == 'left_only'")
    .drop(columns="_merge")
    .assign(pool_changeout_type=lambda x: x["pool_changeout_type"].fillna("P"))
)
new_cc_df

,equipo,componente,subcomponente,position,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type
3,863,CMS,MASA,DERECHO,#LNWB01,2024-W22,2024-06-02,7552.0,15000.0,cms,P
4,863,CMS,Freno Servicio Delanteros,DERECHO,#213,2024-W22,2024-06-02,7552.0,15000.0,cms,P
8,861,CMS,MASA,IZQUIERDO,#PLP0031,2024-W23,2024-06-08,12340.0,15000.0,cms,P
9,861,CMS,Freno Servicio Delanteros,IZQUIERDO,#106,2024-W23,2024-06-08,12340.0,15000.0,cms,P
11,870,Módulo_Potencia,Alternador Principal,UNICO,#EE14060746,2024-W24,2024-06-10,10460.0,16500.0,mp,P
12,870,Módulo_Potencia,SUBFRAME,UNICO,#SF960E862,2024-W24,2024-06-10,10460.0,16500.0,mp,P
13,870,Módulo_Potencia,Radiador,UNICO,#214728-4,2024-W24,2024-06-10,10460.0,16500.0,mp,P
15,852,CMS,MASA,IZQUIERDO,#LNSL17,2024-W24,2024-06-11,8478.0,15000.0,cms,P
16,852,CMS,Freno Servicio Delanteros,IZQUIERDO,#136,2024-W24,2024-06-11,8478.0,15000.0,cms,P
17,855,Módulo_Potencia,Alternador Principal,UNICO,#EE14010714,2024-W24,2024-06-12,8226.0,16500.0,mp,P


In [41]:
cc_df.loc[
    (cc_df["component_code"] == "cd")
    & (cc_df["changeout_week"].isin(["2024-W6", "2024-W7", "2024-W5"]))
]

,equipo,componente,subcomponente,position,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type
2478,870,Cilindro_Dirección,Cilindro Dirección,DERECHO,#EMYK00487,2024-W7,2024-02-12,5231.0,12000.0,cd,None
2479,870,Cilindro_Dirección,Cilindro Dirección,IZQUIERDO,#EMWH340,2024-W7,2024-02-12,3827.0,12000.0,cd,None


In [40]:
cc_df.loc[(cc_df["component_code"] == "cd") & (cc_df["equipo"] == "882")]

,equipo,componente,subcomponente,position,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type
628,882,Cilindro_Dirección,Cilindro Dirección,IZQUIERDO,#EMVW318,2020-W10,2020-03-07,12980.0,12000.0,cd,None
629,882,Cilindro_Dirección,Cilindro Dirección,DERECHO,#EMWB290,2020-W10,2020-03-07,12980.0,12000.0,cd,None
894,882,Cilindro_Dirección,Cilindro Dirección,IZQUIERDO,#EMWG300,2020-W35,2020-08-30,2867.0,12000.0,cd,None
1023,882,Cilindro_Dirección,Cilindro Dirección,DERECHO,#EMWL370,2020-W49,2020-12-03,4293.0,12000.0,cd,None
1840,882,Cilindro_Dirección,Cilindro Dirección,DERECHO,#26KF32,2022-W32,2022-08-14,9717.0,12000.0,cd,None
1980,882,Cilindro_Dirección,Cilindro Dirección,IZQUIERDO,#EMXK390,2022-W48,2022-11-30,13428.0,12000.0,cd,None
2055,882,Cilindro_Dirección,Cilindro Dirección,DERECHO,#EMXM442,2023-W4,2023-01-28,2648.0,12000.0,cd,None
2433,882,Cilindro_Dirección,Cilindro Dirección,DERECHO,#26KF32,2024-W1,2024-01-01,5677.0,12000.0,cd,None
2454,882,Cilindro_Dirección,Cilindro Dirección,IZQUIERDO,#EMXK392,2024-W4,2024-01-26,6985.0,12000.0,cd,None


In [42]:
pool_slots_df.loc[
    (pool_slots_df["component_code"] == "cd")
    & (pool_slots_df["changeout_week"].isin(["2024-W6", "2024-W7", "2024-W5"]))
]

,pool_slot,changeout_week,equipo,component_serial,arrival_week,arrival_date,component_code,componente,subcomponente,position,changeout_date,component_hours,tbo_hours,pool_changeout_type
67,7,2024-W6,870,#EMWH340,2024-W20,2024-05-13,cd,NaN,NaN,NaN,NaT,NaN,NaN,P
68,6,2024-W6,870,#EMYK00487,2024-W20,2024-05-13,cd,NaN,NaN,NaN,NaT,NaN,NaN,P


In [16]:
cc_df.loc[
    (cc_df["component_code"] == "bp") & (cc_df["component_serial"] == "#EE06040239")
]
# & (cc_df['equipo']=='865')

,equipo,componente,subcomponente,position,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type
2411,867,Blower,Blower,DERECHO,#EE06040239,2023-W50,2023-12-15,24109.0,20000.0,bp,None


In [31]:
pool_slots_df.loc[pool_slots_df["changeout_date"].isnull()]

,pool_slot,changeout_week,equipo,component_serial,arrival_week,arrival_date,component_code,componente,subcomponente,position,changeout_date,component_hours,tbo_hours,pool_changeout_type
55,10,2023-W34,858,#73LB-36,2023-W41,2023-10-09,cd,NaN,NaN,NaN,NaT,NaN,NaN,P
63,2,2023-W52,882,#26KF32,2024-W14,2024-04-01,cd,NaN,NaN,NaN,NaT,NaN,NaN,P
67,7,2024-W6,870,#EMWH340,2024-W20,2024-05-13,cd,NaN,NaN,NaN,NaT,NaN,NaN,P
68,6,2024-W6,870,#EMYK00487,2024-W20,2024-05-13,cd,NaN,NaN,NaN,NaT,NaN,NaN,P
93,12,2024-W20,873,#186,2024-W31,2024-07-29,cl,NaN,NaN,NaN,NaT,NaN,NaN,P
94,13,2024-W20,873,#186,2024-W31,2024-07-29,cl,NaN,NaN,NaN,NaT,NaN,NaN,P
134,3,2024-W20,852,#153,2024-W38,2024-09-16,cms,NaN,NaN,NaN,NaT,NaN,NaN,I
139,8,2024-W24,861,#195,2024-W42,2024-10-14,cms,NaN,NaN,NaN,NaT,NaN,NaN,I
142,1,2023-W27,858,#66303255,2023-W48,2023-11-27,mp,NaN,NaN,NaN,NaT,NaN,NaN,I
148,7,2023-W45,399,#66303600,2024-W13,2024-03-25,mp,NaN,NaN,NaN,NaT,NaN,NaN,I


In [68]:
def add_arrival_date(df: pd.DataFrame):
    df = df.assign(
        ohv_normal=df["component_code"].map(
            lambda x: {
                "bp": 51,
                "cd": 46,
                "st": 65,
                "cms": 64,
                "mt": 74,
                "cl": 75,
                "mp": 64,
            }[x]
        ),
        ohv_unplanned=df["component_code"].map(
            lambda x: {
                "bp": 101,
                "cd": 96,
                "st": 125,
                "cms": 124,
                "mt": 134,
                "cl": 135,
                "mp": 114,
            }[x]
        ),
    )
    df = df.assign(
        arrival_date=df["changeout_date"]
        + np.where(
            df["pool_changeout_type"] == "P",
            pd.to_timedelta(df["ohv_normal"], "D"),
            pd.to_timedelta(df["ohv_unplanned"], "D"),
        )
    )
    df = df.assign(arrival_week=df["arrival_date"].dt.strftime("%G-W%V")).drop(
        columns=["ohv_normal", "ohv_unplanned"]
    )
    return df


def find_available_pool_slot(pool_slots_df, changeout_date):
    # Group by pool_slot and find the latest event for each
    latest_events = (
        pool_slots_df.groupby("pool_slot")[
            ["pool_slot", "changeout_date", "arrival_date"]
        ]
        .apply(
            lambda x: x.loc[
                (
                    x["changeout_date"].idxmax()
                    if not x["changeout_date"].isna().all()
                    else x["arrival_date"].idxmax()
                )
            ]
        )
        .reset_index(drop=True)
    )

    # Filter for available slots
    available_slots = latest_events[
        (
            (latest_events["changeout_date"] < changeout_date)
            & (latest_events["arrival_date"] < changeout_date)
        )
        | (latest_events["arrival_date"] < changeout_date)
    ]

    if available_slots.empty:
        return None

    return available_slots  # .loc[available_slots["days_unchanged"].idxmax()]


def find_most_time_unchanged_slot(df, changeout_date):
    # Find the slot with the most time unchanged
    df["days_unchanged"] = (changeout_date - df["arrival_date"]).dt.days
    return df.loc[df["days_unchanged"].idxmax()]


def allocate_components(cc_df, pool_slots_df):
    # Sort cc_df by changeout_date
    cc_df = cc_df.sort_values("changeout_date")

    for _, changeout in cc_df.iterrows():
        available_slots_df = find_available_pool_slot(
            pool_slots_df, changeout["changeout_date"]
        )
        print(changeout)
        available_slot = find_most_time_unchanged_slot(
            available_slots_df, changeout["changeout_date"]
        )
        if available_slot is None:
            print(
                f"Unable to find an available pool slot for changeout on {changeout['changeout_date']}"
            )
            # return pool_slots_df
        # Add new row to pool_slots_df
        new_row = changeout.copy()
        new_row["pool_slot"] = available_slot["pool_slot"]
        new_row["arrival_date"] = pd.NaT  # This will be set when the component returns

        new_row = pd.DataFrame([new_row])
        new_row = add_arrival_date(new_row)

        pool_slots_df = pd.concat([pool_slots_df, new_row], ignore_index=True)

        # Sort the dataframe to maintain chronological order
        pool_slots_df = pool_slots_df.sort_values("changeout_date")
    return pool_slots_df


df = allocate_components(
    new_cc_df.loc[new_cc_df["component_code"].isin(["bp", "mt"])],
    pool_slots_df.loc[pool_slots_df["component_code"].isin(["bp", "mt"])],
)

equipo                                 322
componente                          Blower
subcomponente                       Blower
position                         IZQUIERDO
component_serial               #EN12040115
changeout_week                    2024-W28
changeout_date         2024-07-09 00:00:00
component_hours                     4490.0
tbo_hours                          20000.0
component_code                          bp
pool_changeout_type                      P
Name: 36, dtype: object
equipo                                 320
componente                          Blower
subcomponente                       Blower
position                         IZQUIERDO
component_serial               #EE13101272
changeout_week                    2024-W29
changeout_date         2024-07-15 00:00:00
component_hours                    13291.0
tbo_hours                          20000.0
component_code                          bp
pool_changeout_type                      P
Name: 42, dtype: object
equipo

,pool_slot,changeout_week,equipo,component_serial,arrival_week,arrival_date,component_code,componente,subcomponente,position,changeout_date,component_hours,tbo_hours,pool_changeout_type
0,1,2023-W27,856,#EG08090003,2023-W40,2023-10-02,bp,Blower,Blower,DERECHO,2023-07-03,5726.0,20000.0,P
1,2,2023-W27,881,#EN12040115,2023-W34,2023-08-21,bp,Blower,Blower,IZQUIERDO,2023-07-05,24180.0,20000.0,P
2,3,2023-W27,875,#EN11110073,2023-W38,2023-09-18,bp,Blower,Blower,IZQUIERDO,2023-07-09,23790.0,20000.0,P
3,4,2023-W28,883,#EN12090007,2023-W38,2023-09-18,bp,Blower,Blower,DERECHO,2023-07-14,17676.0,20000.0,P
4,5,2023-W29,870,#EE14010117,2023-W42,2023-10-16,bp,Blower,Blower,DERECHO,2023-07-17,16263.0,20000.0,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,13,2024-W30,398,#EE16030252,2024-W37,2024-09-14,bp,Blower,Blower,DERECHO,2024-07-25,7398.0,20000.0,P
70,1,2024-W30,849,#WX15030003T,2024-W41,2024-10-07,mt,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,2024-07-25,23208.0,24000.0,P
71,2,2024-W30,871,#EE13111400,2024-W38,2024-09-16,bp,Blower,Blower,DERECHO,2024-07-27,18412.0,20000.0,P
72,8,2024-W31,285,#W08031004,2024-W41,2024-10-11,mt,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,2024-07-29,28605.0,24000.0,P


In [60]:
new_cc_df

,equipo,componente,subcomponente,position,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type
3,863,CMS,MASA,DERECHO,#LNWB01,2024-W22,2024-06-02,7552.0,15000.0,cms,P
4,863,CMS,Freno Servicio Delanteros,DERECHO,#213,2024-W22,2024-06-02,7552.0,15000.0,cms,P
7,322,Motor_Tracción,Motor Eléctrico,DERECHO,#EE08030444,2024-W23,2024-06-06,6620.0,24000.0,mt,P
8,322,Motor_Tracción,Freno Servicio Trasero,DERECHO,#358,2024-W23,2024-06-06,6620.0,24000.0,mt,P
9,322,Motor_Tracción,Freno Estacionamiento,DERECHO,#1634,2024-W23,2024-06-06,6620.0,24000.0,mt,P
...,...,...,...,...,...,...,...,...,...,...,...
79,850,CMS,Freno Servicio Delanteros,DERECHO,#162,2024-W31,2024-08-04,4724.0,15000.0,cms,P
80,399,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#W11110015,2024-W32,2024-08-05,1859.0,24000.0,mt,P
81,399,Motor_Tracción,Motor Eléctrico,IZQUIERDO,#EE21040929,2024-W32,2024-08-05,1859.0,24000.0,mt,P
82,399,Motor_Tracción,Freno Servicio Trasero,IZQUIERDO,#166-1,2024-W32,2024-08-05,1859.0,24000.0,mt,P


In [33]:
25 * 0.7

17.5

In [108]:
new_cc_df

,equipo,componente,subcomponente,position,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type
1,872,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#WX1902Y002T,2024-W29,2024-07-15,27730.0,24000.0,mt,None
2,852,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,#WX1410049T,2024-W29,2024-07-21,12339.0,24000.0,mt,None
3,849,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#WX15030003T,2024-W30,2024-07-25,23208.0,24000.0,mt,None
4,285,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,#W08031004,2024-W31,2024-07-29,28605.0,24000.0,mt,None


In [107]:
new_cc_df.iloc[0:1]

,equipo,componente,subcomponente,position,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type
1,872,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#WX1902Y002T,2024-W29,2024-07-15,27730.0,24000.0,mt,None


In [71]:
df = pd.merge(
    pool_proj_df,
    cc_df,
    on=["equipo", "component_code", "component_serial", "changeout_week"],
    how="left",
    suffixes=("", "_proj"),
)
# Recuperar tipo de evento de cambio de componente
df = df.assign(pool_changeout_type=None)
df = df.assign(
    pool_changeout_type=np.where(
        df["pool_changeout_type"].isnull(),
        df["pool_changeout_type_proj"],
        df["pool_changeout_type"],
    ),
)
df = df.assign(
    ohv_normal=df["component_code"].map(
        lambda x: {
            "bp": 51,
            "cd": 46,
            "st": 65,
            "cms": 64,
            "mt": 74,
            "cl": 75,
            "mp": 64,
        }[x]
    ),
    ohv_unplanned=df["component_code"].map(
        lambda x: {
            "bp": 101,
            "cd": 96,
            "st": 125,
            "cms": 124,
            "mt": 134,
            "cl": 135,
            "mp": 114,
        }[x]
    ),
)
df = df.assign(
    proj_arrival_date=df["changeout_date"]
    + np.where(
        df["pool_changeout_type"] == "P",
        pd.to_timedelta(df["ohv_normal"], "D"),
        pd.to_timedelta(df["ohv_unplanned"], "D"),
    )
)

df = df.assign(pool_changeout_type=df["pool_changeout_type"].fillna("P"))
df = df.loc[df["component_code"] == "bp"]
df

,equipo,componente,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type,pool_slot_archived,arrival_week_archived,pool_changeout_type_proj,arrival_date_archived,ohv_normal,ohv_unplanned,proj_arrival_date
4,860,Blower,#EE13111400,2019-W1,2019-01-01,26007.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-12
9,866,Blower,#EE07040188,2019-W2,2019-01-10,12558.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-21
20,869,Blower,#EE14010117,2019-W3,2019-01-17,2577.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-28
21,852,Blower,#EN12100128,2019-W3,2019-01-17,10476.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-28
39,862,Blower,#EN12070112,2019-W6,2019-02-05,14267.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-05-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2633,879,Blower,#EE09100668,2024-W27,2024-07-01,3148.0,20000.0,bp,P,5,2024-W34,P,2024-08-19,51,101,2024-08-21
2637,322,Blower,#EN12040115,2024-W28,2024-07-09,4490.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-10-18
2643,320,Blower,#EE13101272,2024-W29,2024-07-15,13291.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-10-24
2660,398,Blower,#EE16030252,2024-W30,2024-07-25,7398.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-11-03


In [72]:
df = pd.merge_asof(
    df.sort_values("proj_arrival_date"),
    arrivals_df.sort_values("arrival_date"),
    by="component_code",
    left_on="proj_arrival_date",
    right_on="arrival_date",
    direction="nearest",
).sort_values("changeout_date", ascending=False)
df

,equipo,componente,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type,pool_slot_archived,arrival_week_archived,pool_changeout_type_proj,arrival_date_archived,ohv_normal,ohv_unplanned,proj_arrival_date,arrival_week,n_components,pool_slot,arrival_date
238,871,Blower,#EE13111400,2024-W30,2024-07-27,18412.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-11-05,2024-W31,1,0,2024-08-04
237,398,Blower,#EE16030252,2024-W30,2024-07-25,7398.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-11-03,2024-W31,1,0,2024-08-04
236,320,Blower,#EE13101272,2024-W29,2024-07-15,13291.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-10-24,2024-W31,1,0,2024-08-04
235,322,Blower,#EN12040115,2024-W28,2024-07-09,4490.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-10-18,2024-W31,1,0,2024-08-04
233,879,Blower,#EE09100668,2024-W27,2024-07-01,3148.0,20000.0,bp,P,5,2024-W34,P,2024-08-19,51,101,2024-08-21,2024-W31,1,0,2024-08-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,862,Blower,#EN12070112,2019-W6,2019-02-05,14267.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-05-17,2022-W45,1,0,2022-11-06
2,869,Blower,#EE14010117,2019-W3,2019-01-17,2577.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-28,2022-W45,1,0,2022-11-06
3,852,Blower,#EN12100128,2019-W3,2019-01-17,10476.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-28,2022-W45,1,0,2022-11-06
1,866,Blower,#EE07040188,2019-W2,2019-01-10,12558.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-21,2022-W45,1,0,2022-11-06


In [27]:
cc_df

,equipo,componente,SUB COMPONENTE,MODÉLO,POSICION,FECHA DE CAMBIO,cc_week,HORA EQ,HORA CC,MENOR AL 90% TBO,...,Equipo,Solicitud de montaje,OS,RMCARE,CLASIFICACIÓN,COMENTARIO ENTRANTE,COMENTARIO SALIENTE,Proyeccion Horómetro,% uso Proyectado,component
0,862,Motor_Tracción,MOTOR TRACCIÓN,960E-2,IZQUIERDO,2019-01-01,2019.0-W01,25516,14025.0,0.0,...,200159313,56542,NaN,Cerrado,NaN,NaN,NaN,34646.0,1.443583,mt
1,862,Motor_Tracción,Motor Eléctrico,960E-2,IZQUIERDO,2019-01-01,2019.0-W01,25516,14025.0,0.0,...,200214529,56543,NaN,Cerrado,NaN,NaN,NaN,34646.0,1.443583,mt
2,862,Motor_Tracción,Freno Servicio Trasero,960E-2,IZQUIERDO,2019-01-01,2019.0-W01,25516,14025.0,0.0,...,200170908,56660,NaN,Cerrado,NaN,NaN,NaN,34646.0,1.443583,mt
3,862,Motor_Tracción,Freno Estacionamiento,960E-2,IZQUIERDO,2019-01-01,2019.0-W01,25516,14025.0,0.0,...,200174141,56544,NaN,Cerrado,NaN,NaN,NaN,34646.0,1.443583,mt
4,860,Blower,Blower,960E-2,IZQUIERDO,2019-01-01,2019.0-W01,26904,26007.0,1.0,...,200217130,56541,NaN,Cerrado,NaN,NaN,NaN,34646.0,1.732300,bp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2712,285,Motor_Tracción,Freno Servicio Trasero,930E-4,DERECHO,2024-07-29,2024.0-W31,18837,28605.0,1.0,...,200342417,90355,18128007,Cerrado,TBO,NaN,NaN,34.0,0.001417,mt
2713,285,Motor_Tracción,Freno Estacionamiento,930E-4,DERECHO,2024-07-29,2024.0-W31,18837,28605.0,1.0,...,200214225,90356,18128008,Cerrado,TBO,NaN,NaN,34.0,0.001417,mt
2714,847,Cilindro_Dirección,Cilindro Dirección,960E-2,DERECHO,2024-07-29,2024.0-W31,73983,13681.0,1.0,...,200195258,90352,18128009,Cerrado,TBO,NaN,NaN,34.0,0.002833,cd
2715,397,Cilindro_Dirección,Cilindro Dirección,960E-2,IZQUIERDO,2024-07-31,2024.0-W31,69973,13673.0,1.0,...,200210295,90238,NaN,Abierto,TBO,NaN,NaN,0.0,0.000000,cd


In [46]:
pool_proj_df

,pool_slot,changeout_week,equipo,component_serial,changeout_date,arrival_week,pool_changeout_type,arrival_date,component
0,1,2023-W27,856,#EG08090003,2023-07-03,2023-W40,P,2023-10-02,bp
1,2,2023-W27,881,#EN12040115,2023-07-03,2023-W34,P,2023-08-21,bp
2,3,2023-W27,875,#EN11110073,2023-07-03,2023-W38,P,2023-09-18,bp
3,4,2023-W28,883,#EN12090007,2023-07-10,2023-W38,P,2023-09-18,bp
4,6,2023-W29,883,#EE06050200-1,2023-07-17,2023-W42,P,2023-10-16,bp
...,...,...,...,...,...,...,...,...,...
11,12,2024-W11,848,#156,2024-03-11,2024-W20,P,2024-05-13,st
12,1,2024-W16,869,#696R009,2024-04-15,2024-W34,P,2024-08-19,st
13,2,2024-W18,882,#RLTF718,2024-04-29,2024-W36,P,2024-09-02,st
14,3,2024-W19,882,#RLTA681,2024-05-06,2024-W37,P,2024-09-09,st


In [ ]:
df = pd.merge_asof(
    cc_df.sort_values("changeout_date"),
    pool_proj_df.sort_values("changeout_date"),
    by="changeout_date",
    on=["equipo", "component", "component_serial"],
)
# df = df.assign(pool_type=np.where(df["HORA CC"] > 0.9 * df["TBO"], "P", "I"))
df = df.assign(
    ohv_normal=df["componente"].map(
        lambda x: {
            "Blower": 51,
            "Cilindro Dirección": 46,
            "Suspensión Trasera": 65,
            "CMSD": 64,
            "Motor Tracción": 74,
            "Cilindro Levante": 75,
            "Módulo Potencia": 64,
        }[x]
    ),
    ohv_unplanned=df["componente"].map(
        lambda x: {
            "Blower": 101,
            "Cilindro Dirección": 96,
            "Suspensión Trasera": 125,
            "CMSD": 124,
            "Motor Tracción": 134,
            "Cilindro Levante": 135,
            "Módulo Potencia": 114,
        }[x]
    ),
    # cc_date=df["cc_week"].map(lambda x: datetime.strptime(x + "-1", "%Y-W%W-%w")),
)
# df = df.assign(
#     arrival_date=df["cc_date"]
#     + np.where(
#         df["pool_type"] == "P",
#         pd.to_timedelta(df["ohv_normal"], "D"),
#         pd.to_timedelta(df["ohv_unplanned"], "D"),
#     )
# )
df
# df.to_csv("planification-consolidated.csv", index=False)

In [52]:
cc_df

,EQUIPO,COMPONENTE,SUB COMPONENTE,MODÉLO,POSICION,FECHA DE CAMBIO,W,HORA EQ,HORA CC,MENOR AL 90% TBO,...,N/S INSTALADO,Equipo,Solicitud de montaje,OS,RMCARE,CLASIFICACIÓN,COMENTARIO ENTRANTE,COMENTARIO SALIENTE,Proyeccion Horómetro,% uso Proyectado
0,TKD 862,Motor_Tracción,MOTOR TRACCIÓN,960E-2,IZQUIERDO,2019-01-01,1,25516,14025.0,0.0,...,#W13060060,200159313,56542,NaN,Cerrado,NaN,NaN,NaN,34612.0,1.442167
1,TKD 862,Motor_Tracción,Motor Eléctrico,960E-2,IZQUIERDO,2019-01-01,1,25516,14025.0,0.0,...,#EE14080521,200214529,56543,NaN,Cerrado,NaN,NaN,NaN,34612.0,1.442167
2,TKD 862,Motor_Tracción,Freno Servicio Trasero,960E-2,IZQUIERDO,2019-01-01,1,25516,14025.0,0.0,...,#154,200170908,56660,NaN,Cerrado,NaN,NaN,NaN,34612.0,1.442167
3,TKD 862,Motor_Tracción,Freno Estacionamiento,960E-2,IZQUIERDO,2019-01-01,1,25516,14025.0,0.0,...,#470,200174141,56544,NaN,Cerrado,NaN,NaN,NaN,34612.0,1.442167
4,TKD 860,Blower,Blower,960E-2,IZQUIERDO,2019-01-01,1,26904,26007.0,1.0,...,#EE13101129,200217130,56541,NaN,Cerrado,NaN,NaN,NaN,34612.0,1.730600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4906,NaN,_,NaN,NaN,NaN,NaT,52,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4907,NaN,_,NaN,NaN,NaN,NaT,52,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4908,NaN,_,NaN,NaN,NaN,NaT,52,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4909,NaN,_,NaN,NaN,NaN,NaT,52,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df = pd.merge(archived_pool_df, cc_df, on=["pool_slot", "component"], how="left")
df

KeyError: 'pool_slot'

In [8]:
df

,pool_slot,changeout_week,equipo,ns,changeout_date,arrival_week,pool_changeout_type,arrival_date,component
0,1,2023-W28,874,#240,2023-07-10,2023-W45,I,2023-11-06,cms
1,2,2023-W30,875,#201-1,2023-07-24,2023-W46,I,2023-11-13,cms
2,4,2023-W31,859,#189,2023-07-31,2023-W49,I,2023-12-04,cms
3,3,2023-W31,854,#223,2023-07-31,2023-W49,I,2023-12-04,cms
4,5,2023-W34,858,#196,2023-08-21,2023-W42,P,2023-10-16,cms
5,6,2023-W35,878,#221,2023-08-28,2023-W43,P,2023-10-23,cms
6,7,2023-W36,398,#203,2023-09-04,2023-W45,P,2023-11-06,cms
7,8,2023-W37,859,#220,2023-09-11,2023-W46,P,2023-11-13,cms
8,9,2023-W39,856,#228,2023-09-25,2024-W2,I,2024-01-08,cms
9,10,2023-W40,875,#005,2023-10-02,2024-W3,I,2024-01-15,cms
